In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd

## ワインのデータセットの読み込み

In [4]:
wine = load_wine()

In [5]:
pd.DataFrame(wine.data, columns=wine.feature_names)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [6]:
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [7]:
# テストデータ分割
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.25)

In [8]:
len(train_X)

97

In [9]:
len(test_X)

33

## PyTorchのテンソルへ変換

In [10]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

In [11]:
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

In [12]:
train_X.shape

torch.Size([97, 13])

In [13]:
train_Y.shape

torch.Size([97])

In [14]:
train = TensorDataset(train_X, train_Y)

In [15]:
train[0]

(tensor([1.3820e+01, 1.7500e+00, 2.4200e+00, 1.4000e+01, 1.1100e+02, 3.8800e+00,
         3.7400e+00, 3.2000e-01, 1.8700e+00, 7.0500e+00, 1.0100e+00, 3.2600e+00,
         1.1900e+03]), tensor(0))

In [16]:
train_loader = DataLoader(train, batch_size=15, shuffle=True)

## モデルの定義

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [18]:
model = Net()

## トレーニングの実行

In [19]:
criterion = nn.CrossEntropyLoss()

In [20]:
# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=0.001) # 学習率を変更した


In [21]:
for epoch in range(500):
#for epoch in range(2000):
    total_loss = 0
    
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad() # 勾配の初期化
        output = model(train_x) # 出力の計算
        loss = criterion(output, train_y) # 損失の計算
        loss.backward() # 後方に伝播
        optimizer.step() # 伝播結果を更新
        total_loss += loss.item() # トータルロスを加算
        
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

60 1.9728735042735934
120 1.0850520730018616
180 0.7882488407194614
240 0.6908132564276457
300 1.0332058668136597
360 0.7909054942429066
420 0.7202904652804136
480 0.8115143179893494


In [22]:
## 128: 480 4.852030992507935


## 精度の計算

In [23]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

In [24]:
accuracy

0.9090909090909091